In [50]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [51]:
import uuid
bucket_name = 'bankapplication-' + str(uuid.uuid4())  # This will generate a unique bucket name
my_region = boto3.session.Session().region_name
print(f"Region: {my_region}")
print(f"Bucket Name: {bucket_name}")

Region: eu-north-1
Bucket Name: bankapplication-b7c02fe6-e9dc-4369-9db5-4d612b7ad56b


In [52]:
import boto3

s3 = boto3.resource('s3')
try:
    if my_region == 'eu-north-1':
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': my_region
            }
        )
        print('S3 bucket created successfully')
except Exception as e:
    print('Failed to create:', e)


S3 bucket created successfully


In [53]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)


s3://bankapplication-b7c02fe6-e9dc-4369-9db5-4d612b7ad56b/xgboost-as-a-built-in-algo/output


In [54]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print("loading Successfull")
except Exception as e:
    print("data load error")


loading Successfull


In [55]:
try:
    model_data = pd.read_csv("./bank_clean.csv",index_col = 0)
    print("Success load data into fram")
except Exception as e:
    print("error in loading of data into fram")
    

Success load data into fram


In [56]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)



(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [57]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('train.csv', header=False, index=False)
pd.concat([test_data['y_yes'], test_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('test.csv', header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data='s3://{}/{}/train'.format(bucket_name, prefix),
    content_type='csv'
)
s3_input_test = sagemaker.inputs.TrainingInput(
    s3_data='s3://{}/{}/test'.format(bucket_name, prefix),
    content_type='csv'
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [58]:
container = get_image_uri(boto3.Session().region_name,'xgboost',repo_version = '1.0-1')


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [59]:
hyperparameter = {
    "max_depth":"5",
    "eta": "0.2",
    "gamma" : "0.4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic"
   
}

In [60]:
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    hyperparameters=hyperparameter,
    role=sagemaker.get_execution_role(),
    train_instance_count=1,
    train_instance_type="ml.m5.2xlarge",  # Corrected instance type format
    train_volume_size=5,  # Size in GB
    output_path=output_path,
    train_use_spot_instances=True,
    train_max_run=300,
    train_max_wait=600  # Max wait time for spot instances
)
estimator.set_hyperparameters(
    num_round=100,  # Example: Set to a suitable number based on your dataset
    objective="binary:logistic"
)



See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [61]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-02-12-15-37-396


2024-10-02 12:15:40 Starting - Starting the training job...
2024-10-02 12:15:53 Starting - Preparing the instances for training...
2024-10-02 12:16:29 Downloading - Downloading the training image...
2024-10-02 12:17:05 Training - Training image download completed. Training in progress...[2024-10-02 12:17:20.726 ip-10-0-160-192.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single 

In [65]:
xdb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-02-12-32-49-431
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-10-02-12-32-49-431
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-10-02-12-32-49-431


-----!

In [69]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xdb_predictor.serializer = CSVSerializer()
xdb_predictor.content_type = 'text/csv'
predictions = xdb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)


(12357,)


In [70]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10754)    38% (182)
Purchase        9% (1120)     62% (301) 



In [77]:
import boto3
from sagemaker import Session

# Replace with your actual predictor variable (ensure it's correctly defined)
# Assuming xdb_predictor is defined and you want to delete its endpoint
try:
    # Delete the SageMaker endpoint
    sagemaker.Session().delete_endpoint(xdb_predictor.endpoint)
    print(f"Deleted SageMaker endpoint: {xdb_predictor.endpoint}")
except Exception as e:
    print(f"Error deleting endpoint: {e}")

# Replace 'bucket_name' with your actual S3 bucket name
bucket_name = 'bankapplication-b7c02fe6-e9dc-4369-9db5-4d612b7ad56b'

# Initialize the S3 resource
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)

# Delete all objects in the bucket
try:
    # Delete all objects in the bucket
    bucket_to_delete.objects.all().delete()
    print("Deleted all objects in the bucket.")
except Exception as e:
    print(f"Error deleting objects in bucket: {e}")


ClientError: An error occurred (AllAccessDisabled) when calling the ListObjects operation: All access to this object has been disabled